In [19]:
from micrograd.engine import Tensor
import micrograd.nn as nn
import micrograd.functional as F
from micrograd.optim import SGD
from micrograd.data import MnistDataset, DataLoader
import numpy as np
from tqdm import tqdm

MLP definition

In [20]:
class MLP(nn.Module):
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = []
        for i in range(len(nouts)):
            self.layers.append(nn.Linear(sz[i], sz[i+1]))
            if i != len(nouts) - 1:
                self.layers.append(nn.ReLU())

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

Training

In [21]:
train_dataset = MnistDataset(train=True)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [22]:
model = MLP(784, [16, 16, 10])
optimizer = SGD(model.parameters(), lr=0.1)

# training loop
max_epochs = 1
for epoch in range(max_epochs):
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for it, (x, y) in pbar:
        x = x.reshape((-1, 28*28))
        # forward pass
        logits = model(x)
        loss = F.batched_cross_entropy(logits, y)
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # update
        optimizer.step()
        
        pbar.set_description(f"epoch: {epoch}, it: {it}, loss: {loss.data.item():.5f}")

epoch: 0, it: 1874, loss: 0.08704: 100%|██████████████████| 1875/1875 [00:08<00:00, 209.65it/s]


Testing

In [23]:
test_dataset = MnistDataset(train=False)
test_loader = DataLoader(test_dataset, batch_size=128)

In [24]:
# evaluate accuracy on test set
correct = 0
total = 0
for x, y in test_loader:
    x = x.reshape((-1, 28*28))
    logits = model(x)
    pred = np.argmax(logits.data, axis=-1)
    correct += (y.data == pred).sum()
    total += x.data.shape[0]
    

print(f"accuracy: {correct / total}")

accuracy: 0.9231
